In [47]:
!python -m pip install --quiet --upgrade pip
!pip install mindscope_utilities --quiet --upgrade

In [48]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

import mindscope_utilities
import mindscope_utilities.visual_behavior_ophys as ophys

from allensdk.brain_observatory.behavior.behavior_project_cache import VisualBehaviorOphysProjectCache

pd.set_option('display.max_columns', 500)

In [49]:
data_storage_directory = "/temp" # Note: this path must exist on your local drive
cache = VisualBehaviorO  physProjectCache.from_s3_cache(cache_dir=data_storage_directory)

In [59]:
session_table = cache.get_behavior_ophys_experiment(951980486)


In [60]:
# for get_behavior_ophys_experiment
help(session_table)

Help on BehaviorOphysExperiment in module allensdk.brain_observatory.behavior.behavior_ophys_experiment object:

class BehaviorOphysExperiment(allensdk.brain_observatory.behavior.behavior_session.BehaviorSession)
 |  BehaviorOphysExperiment(behavior_session: allensdk.brain_observatory.behavior.behavior_session.BehaviorSession, projections: allensdk.brain_observatory.behavior.data_objects.projections.Projections, ophys_timestamps: allensdk.brain_observatory.behavior.data_objects.timestamps.ophys_timestamps.OphysTimestamps, cell_specimens: allensdk.brain_observatory.behavior.data_objects.cell_specimens.cell_specimens.CellSpecimens, metadata: allensdk.brain_observatory.behavior.data_objects.metadata.behavior_ophys_metadata.BehaviorOphysMetadata, motion_correction: allensdk.brain_observatory.behavior.data_objects.motion_correction.MotionCorrection, eye_tracking_table: Union[allensdk.brain_observatory.behavior.data_objects.eye_tracking.eye_tracking_table.EyeTrackingTable, NoneType], eye_tra

In [58]:
# for get_behavior_session
help(session_table)

Help on BehaviorSession in module allensdk.brain_observatory.behavior.behavior_session object:

class BehaviorSession(allensdk.brain_observatory.behavior.data_objects.base._data_object_abc.DataObject, allensdk.brain_observatory.behavior.data_objects.base.readable_interfaces.LimsReadableInterface, allensdk.brain_observatory.behavior.data_objects.base.readable_interfaces.NwbReadableInterface, allensdk.brain_observatory.behavior.data_objects.base.readable_interfaces.JsonReadableInterface, allensdk.brain_observatory.behavior.data_objects.base.writable_interfaces.NwbWritableInterface)
 |  BehaviorSession(behavior_session_id: allensdk.brain_observatory.behavior.data_objects.metadata.behavior_metadata.behavior_session_id.BehaviorSessionId, stimulus_timestamps: allensdk.brain_observatory.behavior.data_objects.timestamps.stimulus_timestamps.stimulus_timestamps.StimulusTimestamps, running_acquisition: allensdk.brain_observatory.behavior.data_objects.running_speed.running_acquisition.RunningAcqui

In [ ]:
session_table.stimulus

In [ ]:
session_table = cache.get_ophys_session_table()
experiment_table = cache.get_ophys_experiment_table()
behavior_table = cache.get_behavior_session_table()


In [ ]:
ophys_session_id = 952430817

In [ ]:
experiments = {}
ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']
for ophys_experiment_id in ophys_experiment_ids:
    experiments[ophys_experiment_id] = cache.get_behavior_ophys_experiment(ophys_experiment_id)

* Targeted structure is in experiment metadata


In [ ]:
neural_data = []
for ophys_experiment_id in tqdm(experiments.keys()): #tqdm is a package that shows progress bars for items that are iterated over
    this_experiment = experiments[ophys_experiment_id]
    this_experiment_neural_data = ophys.build_tidy_cell_df(this_experiment)
    
    # add some columns with metadata for the experiment
    metadata_keys = [
        'ophys_experiment_id',
        'ophys_session_id',
        'targeted_structure',
        'imaging_depth',
        'equipment_name',
        'cre_line',
        'mouse_id',
        'sex',
    ]
    for metadata_key in metadata_keys:
        this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]
        
    # append the data for this experiment to a list
    neural_data.append(this_experiment_neural_data)
    
# concatate the list of dataframes into a single dataframe
neural_data = pd.concat(neural_data)

100%|██████████| 4/4 [00:03<00:00,  1.28it/s]


In [ ]:
behavior_table.head()

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,ophys_session_id,ophys_experiment_id,ophys_container_id,project_code,date_of_acquisition,session_type,file_id
behavior_session_id,,,,,,,,,,,,,,,,,,,,
870987812,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,80.0,Sst-IRES-Cre,GCaMP6f,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,2019-05-17 14:46:33.550,TRAINING_0_gratings_autorewards_15min,1.085368e+09
872953842,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,84.0,Sst-IRES-Cre,GCaMP6f,NaN,0.0,NaN,0.0,NaN,NaN,NaN,NaN,2019-05-21 13:22:20.613,TRAINING_1_gratings,1.085368e+09
873774125,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,85.0,Sst-IRES-Cre,GCaMP6f,NaN,1.0,NaN,0.0,NaN,NaN,NaN,NaN,2019-05-22 13:19:52.005,TRAINING_1_gratings,1.085368e+09
874790261,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,86.0,Sst-IRES-Cre,GCaMP6f,NaN,2.0,NaN,0.0,NaN,NaN,NaN,NaN,2019-05-23 13:40:31.405,TRAINING_1_gratings,1.085368e+09
875207834,BEH.D-Box1,Sst-IRES-Cre/wt;Ai148(TIT2L-GC6f-ICL-tTA2)/wt,457841,Ai148(TIT2L-GC6f-ICL-tTA2),[Sst-IRES-Cre],F,87.0,Sst-IRES-Cre,GCaMP6f,NaN,3.0,NaN,0.0,NaN,NaN,NaN,NaN,2019-05-24 11:06:39.332,TRAINING_1_gratings,1.085368e+09


In [ ]:
behaviours = {}
ophys_experiment_ids = session_table.loc[ophys_session_id]['ophys_experiment_id']

In [ ]:
sid = [870987812,872953842,873774125,874790261]
behavior_session = cache.get_behavior_session(sid[3])

# behavior_attributes = sorted(list(behavior_session.__dict__.keys()))


behavior_session_874790261.nwb: 100%|██████████| 66.7M/66.7M [00:01<00:00, 34.4MMB/s]


In [ ]:
behavior_session.rewards

In [ ]:
behavior_session.stimulus_presentations['image_name'].unique()

array(['gratings_270.0', 'gratings_0.0', 'gratings_180.0',
       'gratings_90.0'], dtype=object)

In [ ]:
behavior_session.stimulus_timestamps

array([0.00000000e+00, 1.66341281e-02, 3.32983835e-02, ...,
       3.64515840e+03, 3.64517507e+03, 3.64519175e+03])

In [ ]:
len(behavior_session.running_speed)

218527

In [ ]:
behaviours = {}
for ophys_experiment_id in ophys_experiment_ids:
    behaviours[ophys_experiment_id] = cache.get_behavior_ophys_experiment(ophys_experiment_id)
behaviour_data = []


In [ ]:
type(behavior_session)

allensdk.brain_observatory.behavior.behavior_session.BehaviorSession

In [ ]:
help(ophys.add_mean_pupil_area_to_stimulus_presentations)

Help on function add_mean_pupil_area_to_stimulus_presentations in module mindscope_utilities.visual_behavior_ophys.data_formatting:

add_mean_pupil_area_to_stimulus_presentations(stimulus_presentations, eye_tracking, time_window=[-3, 3])
        Append a column to stimulus_presentations which contains
        the mean pupil area in a range relative to
        the stimulus start time.
    
        Args:
            stimulus_presentations (pd.DataFrame): dataframe of stimulus presentations.  # noqa E501
                Must contain: 'start_time'
            eye_tracking (pd.DataFrame): dataframe of eye tracking data.
                Must contain: 'pupil_area', 'timestamps'
            time_window (list with 2 elements): start and end of the range
                relative to the start of each stimulus to average the pupil area.
        Returns:
            stimulus_presentations table with new column "mean_pupil_area" with the
            mean pupil arae within the specified window
      

In [ ]:
# noval sessions
exp = 0.0
fam = 50.0
noval_sessions = session_table.query('prior_exposures_to_image_set == @exp')
familiar_sessions = session_table.query('prior_exposures_to_image_set >= @fam')

In [ ]:
experiment_table['mouse_id'].unique()

array([457841, 476970, 412036, 429952, 449441, 491060, 450471, 448900,
       459777, 467951, 479839, 499478, 456564, 467953, 459773, 464207,
       513626, 447663, 523922, 484408, 533161, 445270, 485689, 555970,
       453989, 453988, 467302, 544261, 489065, 403491, 533527, 538219,
       484631, 546605, 479426, 457766, 523927, 489066, 456917, 464204,
       495789, 547486, 528097, 512458, 470784, 477052, 554115, 451787,
       546819, 485688, 438912, 570949, 456916, 477202, 539518, 435431,
       423606, 524274, 425493, 453991, 481295, 486737, 431023, 476631,
       442709, 563234, 498972, 485152, 544965, 436662, 489056, 492395,
       561443, 449653, 541329, 524158, 440298, 479458, 548950, 476067,
       431252, 484627, 456915, 472271, 462544, 453913, 412366, 462468,
       448366, 539517, 482853, 453990, 513630, 421136, 425496, 440631,
       563231, 445002, 480753, 461946, 453911, 549976, 550603, 547266,
       533162, 467954, 468866])

In [ ]:
mo_id = 479426

In [ ]:
experiments_table = experiment_table.query('mouse_id == @mo_id')

In [ ]:
exp = 0.0
fam = 27.0
noval_sessions = experiments_table.query('prior_exposures_to_image_set == @exp')
familiar_sessions = experiments_table.query('prior_exposures_to_image_set >= @fam')

In [ ]:
noval_sessions

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,ophys_session_id,behavior_session_id,ophys_container_id,project_code,imaging_depth,targeted_structure,date_of_acquisition,session_type,experience_level,passive,image_set,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,
974945686,CAM2P.5,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,479426,Ai94(TITL-GCaMP6s),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,142.0,Slc17a7-IRES2-Cre,GCaMP6s,4.0,0.0,0.0,5.0,974479344,974606492,968451510,VisualBehavior,175,VISp,2019-10-31 15:00:58.000000,OPHYS_4_images_B,Novel 1,False,B,975933665


In [ ]:
familiar_sessions

,equipment_name,full_genotype,mouse_id,reporter_line,driver_line,sex,age_in_days,cre_line,indicator,session_number,prior_exposures_to_session_type,prior_exposures_to_image_set,prior_exposures_to_omissions,ophys_session_id,behavior_session_id,ophys_container_id,project_code,imaging_depth,targeted_structure,date_of_acquisition,session_type,experience_level,passive,image_set,file_id
ophys_experiment_id,,,,,,,,,,,,,,,,,,,,,,,,,
974384470,CAM2P.5,Slc17a7-IRES2-Cre/wt;Camk2a-tTA/wt;Ai94(TITL-G...,479426,Ai94(TITL-GCaMP6s),"[Slc17a7-IRES2-Cre, Camk2a-tTA]",F,141.0,Slc17a7-IRES2-Cre,GCaMP6s,3.0,1.0,27.0,4.0,974019098,974140453,968451510,VisualBehavior,175,VISp,2019-10-30 15:40:18.000000,OPHYS_3_images_A,Familiar,False,A,974487242


In [ ]:
experiments_table['prior_exposures_to_image_set'].unique()

array([23., 26., 27.,  0.,  2.,  3.])

In [ ]:
# neural_data = cache.get_behavior_ophys_experiment(974384470)

In [ ]:
familiar_sessions.keys()

Index(['equipment_name', 'full_genotype', 'mouse_id', 'reporter_line',
       'driver_line', 'sex', 'age_in_days', 'cre_line', 'indicator',
       'session_number', 'prior_exposures_to_session_type',
       'prior_exposures_to_image_set', 'prior_exposures_to_omissions',
       'ophys_session_id', 'behavior_session_id', 'ophys_container_id',
       'project_code', 'imaging_depth', 'targeted_structure',
       'date_of_acquisition', 'session_type', 'experience_level', 'passive',
       'image_set', 'file_id'],
      dtype='object')

In [ ]:
familiar_sessions.index[0]

974384470

In [ ]:
experiments = {}
ophys_experiment_ids = noval_sessions.index
for ophys_experiment_id in ophys_experiment_ids:
    print(ophys_experiment_id)
    experiments[ophys_experiment_id] = cache.get_behavior_ophys_experiment(ophys_experiment_id)

974945686


behavior_ophys_experiment_974945686.nwb:  11%|█         | 252M/2.39G [00:07<00:50, 42.5MMB/s]

KeyboardInterrupt: ignored

In [ ]:
neural_data = []

for ophys_experiment_id in tqdm(familiar_sessions.index): #tqdm is a package that shows progress bars for items that are iterated over
    this_experiment = experiments[ophys_experiment_id]
    this_experiment_neural_data = ophys.build_tidy_cell_df(this_experiment)
    
    # add some columns with metadata for the experiment
    metadata_keys = [
        'ophys_experiment_id',
        'ophys_session_id',
        'targeted_structure',
        'imaging_depth',
        'equipment_name',
        'cre_line',
        'mouse_id',
        'sex',
    ]
    for metadata_key in metadata_keys:
        this_experiment_neural_data[metadata_key] = this_experiment.metadata[metadata_key]
        
    # append the data for this experiment to a list
    neural_data.append(this_experiment_neural_data)
    
# concatate the list of dataframes into a single dataframe
neural_data = pd.concat(neural_data)

  0%|          | 0/1 [00:00<?, ?it/s]

974384470


NameError: ignored